In [1]:
import json
import h5py
import numpy as np
from torch import nn
import torch
from torch import stft
import matplotlib.pyplot as plt 
from matplotlib.colors import LogNorm
import PIL
from tqdm import tqdm
import os
import pandas as pd

from utils.data_utils import load_recording
from utils.data_utils import plot_spec, plot_first_n
from utils.segments_utils import get_valid_sections

In [2]:
data_dir = '/home/ubuntu/small_data/small_data'
all_files = [x for x in os.listdir(data_dir) if '.h5' in x]

In [3]:
l = pd.read_parquet('/home/ubuntu/small_data/small_data/metadata.parquet')[['filename_h5', 'deviceinfo_description_serial']]
l

,filename_h5,deviceinfo_description_serial
24306,link-4d26f93f-3948-4f80-aa89-6959d36a0bd7--c0a...,810774
8027,link-5a3b40ad-e2b1-4feb-82ba-ec511e735ca2--236...,820413
14215,link-08753858-caae-463b-9a9e-5c4f22ab7ad3--2cf...,820081
4955,link-515ca31c-f3dd-43cb-91d9-e8995406f619--eaa...,810778
23921,link-21b0bbe8-295c-41c7-90f6-44126b8eb38a--9f8...,820081
...,...,...
20209,link-78bc393f-71f8-4465-8ad8-0b160ce13556--443...,820045
23721,link-60ae80a5-db55-43fd-af52-58e8daa78f4a--819...,810103
13318,link-afd05d8f-cc08-41c0-9ded-65550b7e254c--2f9...,820314
28437,link-b3f18eda-ebec-4982-bb10-e73c7f654019--565...,820125


In [4]:
from utils.data_utils import band_pass_brickwall

new_path = '/home/ubuntu/small_data/sloped_brickwall'


for _, row in tqdm(l.iterrows(), total=len(l)):
    try:
        data, _, _ = load_recording(data_dir + '/' + row['filename_h5'])
        data = torch.tensor(data, device=torch.device('cuda'))
        data = torch.stack([
            band_pass_brickwall(data[0], 1, 40),
            band_pass_brickwall(data[1], 1, 40),
            band_pass_brickwall(data[2], 1, 40),
            band_pass_brickwall(data[3], 1, 40),
        ])
        data = torch.clip(data, -1.95e-4, 1.95e-4)
        # high pass should remove DC offset, but lets substract mean anyways
        data = data - data.mean(1, keepdim=True)
        # lets normalize variance across all channels, to preserve info between channels
        data = data / ((data ** 2).mean() ** 0.5) # make std 1
    
    
        new_fn = f"{new_path}/{row['filename_h5'][:-3]}_{row['deviceinfo_description_serial']}.pt"
        torch.save(data, new_fn)
    except Exception:
        pass

  0%|          | 0/500 [00:00<?, ?it/s]/home/ubuntu/neural_foundation/src/utils/data_utils.py:113: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1031.)
  mask = torch.nn.functional.conv1d(mask.unsqueeze(0), smooth_kernel, bias=None, stride=1, padding='same')
100%|██████████| 500/500 [02:29<00:00,  3.34it/s]
